In [50]:
from stem import Signal
from stem.control import Controller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from chromedriver_py import binary_path
import hashlib
import codecs
import pprint
import random
import re
import requests
import sys
import time

In [51]:
_GOOGLEID = hashlib.md5(str(random.random()).encode('utf-8')).hexdigest()[:16]
_COOKIES = {'GSP': 'ID={0}:CF=4'.format(_GOOGLEID)}
_HOST = 'https://scholar.google.com'
_AUTHSEARCH = '/citations?view_op=search_authors&hl=en&mauthors={0}'
_CITATIONAUTH = '/citations?user={0}&hl=en'
_CITATIONPUB = '/citations?view_op=view_citation&citation_for_view={0}'
_KEYWORDSEARCH = '/citations?view_op=search_authors&hl=en&mauthors=label:{0}'
_PUBSEARCH = '/scholar?q={0}'
_SCHOLARPUB = '/scholar?oi=bibs&hl=en&cites={0}'

_CITATIONAUTHRE = r'user=([\w-]*)'
_CITATIONPUBRE = r'citation_for_view=([\w-]*:[\w-]*)'
_SCHOLARCITERE = r'gs_ocit\(event,\'([\w-]*)\''
_SCHOLARPUBRE = r'cites=([\w-]*)'
_EMAILAUTHORRE = r'Verified email at '

_SESSION = requests.Session()
_PAGESIZE = 100

In [52]:
def switchIP():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

def get_browser(PROXY = None):
    chrome_options = webdriver.ChromeOptions()
    if PROXY != None:
        chrome_options.add_argument('--proxy-server=SOCKS5://{0}'.format(PROXY)) 
    chrome_options.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度 # don't load 
    chrome_options.add_argument('--headless') #don't launch visibile browser interface
    executable_path='/Users/fewave/project/python/demo/chromedriver' #设置启动驱动
    return webdriver.Chrome(executable_path=binary_path, options=chrome_options)


In [78]:
def get_soup(url):
    switchIP()
    driver = get_browser()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def get_next_page(url):
    soup = get_soup(url)
    for left in soup.findAll('td', {'align':"left"}):
        next_page = _HOST+left.find_all('a', href=True)[0]['href']
    return next_page


def key_word(keyword, max_entry = 15, title_exact_match = False):
    url = _HOST+_PUBSEARCH.format(requests.utils.quote(keyword))
    number_of_entry=0
    counter = 0
    turn_page = True
    entry = {}
    authors = []
    title = []
    abstract = []
    while turn_page == True:
        soup=get_soup(url)

        for row in soup.findAll('div', {'class':'gs_r gs_or gs_scl'}):
            if counter == max_entry:
                break
            author=soup.findAll('div', {'class':'gs_a'})[counter].get_text(strip=True)
            title = soup.findAll('h3', {'class':'gs_rt'})[counter].get_text(strip=True)
            entry[number_of_entry] = {'author': author, 'article':title}
            number_of_entry+=1
            counter +=1

        if counter < max_entry:
            url = get_next_page(url)
            if url == None:
                turn_page = False
            max_entry = max_entry - counter
            counter = 0

        else:
            turn_page = False

    return entry

key_word('Maos China and the Cold war')

{0: {'author': 'J Chen - 2010 - books.google.com',
  'article': "[BOOK][B]Mao's Chinaand thecold war"},
 1: {'author': 'S Zhihua, Z Shen - 2012 - books.google.com',
  'article': '[BOOK][B]Mao, Stalin and the Koreanwar: Trilateral communist relations in the 1950s'},
 2: {'author': 'RS Ross - 2015 - books.google.com',
  'article': '[BOOK][B]China, the United States, and the Soviet Union: tripolarity and policy making in theCold War'},
 3: {'author': 'R Mitter\xa0- Journal of Contemporary History, 2003 - journals.sagepub.com',
  'article': "Old ghosts, new memories:China'schangingwarhistory in the era of post-Maopolitics"},
 4: {'author': 'RS Ross,B Naughton- 1998 - books.google.com',
  'article': '[BOOK][B]After theCold War: Domestic Factors and US-ChinaRelations'},
 5: {'author': 'TJ Christensen- International Security, 1992 - JSTOR',
  'article': "Threats, assurances, and the last chance for peace: The lessons ofMao'sKoreanwartelegrams"},
 6: {'author': 'T Brook, BM Frolic - 1997 - boo

In [77]:
def find_class_name(soup, tag, text):
    elements = soup.find_all(tag)
    for element in elements:
        if 'class' in element.attrs and text in element.attrs['class'][0]:
            return element.attrs['class'][0]
        else:
            return None
        
def scholar_search(name, max_entry = 15):
    url = _HOST+_AUTHSEARCH.format(requests.utils.quote(name))
    
    counter = 0
    turn_page = True
    number_of_entry=0
    counter = 0
    turn_page = True
    entry = {}
    authors = []
    title = []
    abstract = []
    
    while turn_page == True:

        soup = get_soup(url)
        
        for row in soup.find_all('div', 'gsc_1usr'):
            if counter == max_entry:
                break
            author_id = re.findall(_CITATIONAUTHRE, row('a')[0]['href'])[0]
            author_name = soup.find('h3', class_=find_class_name(soup, 'h3', 'name')).text
            for author_info in row.findAll('div', {'class':'gs_ai_t'}):
                citedby=author_info.findAll('div', {'class':'gs_ai_cby'})[0].get_text(strip=True)
                affiliation = author_info.findAll('div', {'class':'gs_ai_aff'})[0].get_text(strip=True)
                feilds = [i.text.strip()  for i in
                           author_info.findAll('a', {'class':'gs_ai_one_int'})]
                
            '''email = soup.find('div', class_=find_class_name(row, 'div', 'eml'))
           
            if email:
                email = re.sub(_EMAILAUTHORRE, r'@', email.text)
            else:
                email = None'''
                
            
            entry[number_of_entry] = {'author_id':author_id, 'name':author_name, 
                                      'affiliation':affiliation, 'email': None, 
                                      'feilds':feilds, 'citedby':citedby}
            number_of_entry+=1
            counter +=1
            
        if counter < max_entry:
            next_page = soup.find(class_='gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')
            if next_page and 'disabled' not in next_page.attrs:
                url = next_page['onclick'][17:-1]
                url = _HOST + codecs.getdecoder("unicode_escape")(url)[0]
                soup = get_soup(url)
            else:
                turn_page = False
            max_entry = max_entry - counter
            counter = 0

        else:
            turn_page = False
    
    return entry
scholar_search('Jian Chen')

{0: {'author_id': 'piDcgOwAAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Associate Professor of Institute of Electronics, Chinese Academy of Sciences',
  'email': None,
  'feilds': ['Microfluidics'],
  'citedby': 'Cited by 37323'},
 1: {'author_id': 'GuLkXX4AAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'University of Leicester',
  'email': None,
  'feilds': ['vibration',
   'acoustics',
   'bionic engineering',
   'machine condition monitoring',
   'and structural health monitoring'],
  'citedby': 'Cited by 19979'},
 2: {'author_id': 'EO2PG-8AAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Chair Professor of Ocean Eng., Southern University of Science and Technology',
  'email': None,
  'feilds': ['Civil Engineering',
   'Structural Engineering',
   'Structures',
   'FRP',
   'DEM'],
  'citedby': 'Cited by 11280'},
 3: {'author_id': 'W6JT9HsAAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Associate Professor, Medical College of Georgia, Augusta University',
  'email': None,
  'feild

https://scholar.google.com/scholar?q=Cultural%20Revolution
